In [ ]:
from profiling.funs import load, my_key, _save
import pandas as pd

In [ ]:
def check(keywords):
    def find(x):
        return any(
            [word in x for word in keywords]
        )
    return find

# TensorFlow

In [ ]:
key_ops = ['sequential', 'gradient_tape', 'categorical_crossentropy', 'SGD']
key_typs = ['Dataset']

def no_idle(df):
    return df[~df['Operation'].apply(check(['IDLE'])) ]

def refresh(df):
    df = no_idle(df)
    df = df.drop(columns=['Total self-time on Host (%)', 'Avg. time (us)', 'Total time (us)'])
    
    return df

def total_op(df, words):
    mask = df['Operation'].apply(check(words))

    return df[mask]['Avg. self-time (us)'].sum()

def from_set(fz):
    return {name:value for (name,value) in fz}

def extract_my_names(key, value):
    dct=from_set(key)
    num=total_op(value, dct['model_str'].upper())
    dct['avg_time_us'] = num
    
    return dct

In [ ]:
def save_ops_total(num):
    df_dict = load('profiling/results/data_{}.tflow'.format(num))
    biglist=[extract_my_names(key, value) for key,value in df_dict.items()]
    data=pd.DataFrame(data=biglist)
    
    _save(data,'datasets/ops_total_{}.tflow'.format(num))

In [ ]:
save_ops_total(1)

In [ ]:
save_ops_total(2)

# PyTorch